<a href="https://colab.research.google.com/github/abs711/nanoGPT/blob/master/nanoGPT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-11-28 01:24:10--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2023-11-28 01:24:10 (23.7 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [8]:
f"Total document length: {len(text)}"

'Total document length: 1115394'

In [13]:
#Unique characters in the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [20]:
#Tokenization

stoi = {ch:i for i, ch in enumerate(chars)}

itos = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("Please encode"))

print(decode(encode("please decode")))

[28, 50, 43, 39, 57, 43, 1, 43, 52, 41, 53, 42, 43]
please decode


In [28]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
f"{data.shape}, {data.dtype}"

'torch.Size([1115394]), torch.int64'

In [29]:
n = int(0.9*len(data))

train_data = data[:n]
test_data = data[n:]

In [40]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8 #context_length

In [51]:
def get_batch(split):
  data = train_data if split == 'train' else torch._validate_sparse_compressed_tensor_args
  ix  = torch.randint(len(data) - block_size, (batch_size, ))
  # print(ix)
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb =  get_batch('train')

print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
# for b in range(batch_size):
#   for t in range(block_size):
#     context = xb[b,:t+1]
#     target = yb[b,t]
#     print(f"Input:{context.tolist()}, target: {target}")

inputs:
torch.Size([32, 8])
tensor([[54, 43, 41, 58, 47, 60, 43, 57],
        [ 1, 39, 40, 56, 53, 39, 42, 12],
        [43,  6,  1, 53, 56,  1, 63, 53],
        [47, 41, 47, 53, 59, 57,  2,  0],
        [56, 53, 61, 57,  1, 59, 52, 58],
        [56, 47, 58,  1, 58, 46, 39, 58],
        [ 1, 61, 46, 53,  1, 63, 53, 59],
        [ 0, 35, 43,  5, 50, 50,  1, 46],
        [ 5, 50, 50,  1, 58, 59, 56, 52],
        [39, 56, 39, 41, 58, 57,  6,  1],
        [ 1, 27,  6,  1, 61, 53, 59, 50],
        [34, 10,  0, 32, 46, 43, 52,  1],
        [39, 56,  1, 58, 46, 47, 57,  1],
        [17, 31, 15, 13, 24, 33, 31, 10],
        [52, 42,  1, 54, 39, 47, 52, 58],
        [43,  1, 41, 53, 50, 53, 59, 56],
        [ 6,  1, 44, 53, 56,  1, 46, 43],
        [ 1, 58, 46, 47, 52, 49,  1, 58],
        [43, 56,  6,  1, 53, 56,  1, 57],
        [61, 43, 50, 50,  1, 40, 56, 47],
        [57, 58, 43, 56,  0, 31, 58, 39],
        [61, 46, 63,  6,  1, 58, 46, 43],
        [58, 46, 53, 59,  1, 61, 43, 43],
      

In [52]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):

    logits =self.token_embedding_table(idx) # B, T, C
    if targets is None:
      loss= None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      # get the predictions
      logits, loss = self(idx)
      # last time step
      logits = logits[:,-1,:]

      probs = F.softmax(logits, dim=-1)

      idx_next = torch.multinomial(probs, num_samples=1)

      idx = torch.cat((idx, idx_next), dim=1)
    return idx


m =  BigramLanguageModel(vocab_size)
logits, loss= m(xb, yb)
print(logits.shape)
print(loss)




print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([256, 65])
tensor(4.7815, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [53]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [56]:
batch_size = 32

for steps in range(10000):
  xb, yb = get_batch('train')

  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.4199717044830322


In [58]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


ANGO:
He rthay n thavee
Sw s serer Fofow.
Houspathe t:
Mind fit.
DUKINoceamy hun.
CKIUShorst onre t ache bar, simed?
And me theluse BHENurind-g'sto f w m CK:
YCESI fatass mbre lious ave
Wer'dor' wod y:

Henkns ges wise we me y to elil'doug p in t her spalisusin t wndalu?Y!

CKINENGLOFrkeang-lumod n odas ine a! thayayor hannd t; frat.
OLArZAUSum,
s I f pin hondecharvyouke helldid t we keicetlot llly ide ply t com: chavily; o fill wf mes co it hitodntly d d wsue ater to ovithA herisch ave une, t e
